In [ ]:
# Copyright 2023 Milla-Val et al.

In [ ]:
from IPython.display import Markdown, display

display(Markdown("README.md"))

## Import TensorFlow and other libraries

In [ ]:
import tensorflow as tf

from tensorflow.keras.callbacks import ModelCheckpoint

import os
import sys
import time
import datetime


import numpy as np
import pandas as pd
import matplotlib.colors as colors

from matplotlib import pyplot as plt
from IPython import display
from tqdm.auto import tqdm

## Auxiliary steps

In [ ]:
# Set memory growth for the GPU to reduce excesice VRAM utilization
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

# Create, if it not exists, a folder
def creaCarpeta(ruta_save_fig='/folder/folder2'):
    try: 
        os.makedirs(ruta_save_fig)
    except OSError:
        if not os.path.isdir(ruta_save_fig):
            raise
    return ruta_save_fig+"/"

## Paths for data inputs, logs, checkpoints and results

In [ ]:
# Path for the input data
data_folder = '/home/jmilla/DoctoradoData/piri10h_datasets/dataPiri1h_resMean.npz'

# Path for the checkpoint directory. 
# Pointing this to a ramdisk speed-up training.
checkpoint_dir = '/home/jmilla/ramdisk/pix2pix_own_ckpt'

# Path for the logs files
log_dir = "logs/"

# Path for results. 'creaCarpeta' will create a new folder with the specified name
res_dir = creaCarpeta('./results')

## Load data

In [ ]:
compNUM = 2

data = np.load(data_folder)

#([Ux(0), Uy(1), Uz(2), u(3), v(4), wa(5)], dates, lat, lon, eta)
velTrain = data['velTrain'] 
velTest = data['velTest']
# print(velTrain.shape, velTest.shape)

cfdTrain = velTrain[0:compNUM,:,:,:,0:1]
cfdTrain = cfdTrain.swapaxes(0,-1)[0]
wrfTrain = velTrain[3:(3+compNUM),:,:,:,0:1]
wrfTrain = wrfTrain.swapaxes(0,-1)[0]

cfdTest = velTest[0:compNUM,:,:,:,0:1]
cfdTest = cfdTest.swapaxes(0,-1)[0]
wrfTest = velTest[3:(3+compNUM),:,:,:,0:1]
wrfTest = wrfTest.swapaxes(0,-1)[0]

display.display(('cfdTrain', cfdTrain.shape), (cfdTrain.max((0,1,2)), cfdTrain.min((0,1,2))))
display.display(('wrfTrain', wrfTrain.shape), (wrfTrain.max((0,1,2)), wrfTrain.min((0,1,2))))
display.display(('cfdTest', cfdTest.shape), (cfdTest.max(), cfdTest.min()))
display.display(('wrfTest', wrfTest.shape), (wrfTest.max(), wrfTest.min()))

In [ ]:
BUFFER_SIZE = 193
BATCH_SIZE = 32
IMG_WIDTH = 128
IMG_HEIGHT = 128

In [ ]:
# Normalizing the images to [-1, 1]
def normalize(inp, real):
    inp = (inp + 30) / 60
    inp = tf.cast(inp*2.0 - 1.0, tf.float32)
    
    real = (real + 60) / 140
    real = tf.cast(real*2.0 - 1.0, tf.float32)

    return inp, real

## Build an input pipeline with `tf.data`

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((wrfTrain, cfdTrain))
test_dataset = tf.data.Dataset.from_tensor_slices((wrfTest, cfdTest))

train_dataset = train_dataset.map(normalize,
                                  num_parallel_calls=tf.data.AUTOTUNE)

train_dataset = train_dataset.shuffle(BUFFER_SIZE, seed=1234)
train_dataset = train_dataset.batch(BATCH_SIZE)
val_dataset = train_dataset.take(int(len(train_dataset)*0.2))
train_dataset = train_dataset.skip(int(len(train_dataset)*0.2))

test_dataset = test_dataset.map(normalize,
                                num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE)

## Build the generator

The generator of the pix2pix cGAN is a _modified_ [U-Net](https://arxiv.org/abs/1505.04597). A U-Net consists of an encoder (downsampler) and decoder (upsampler). (You can find out more about it in the [U-Net project website](https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/).)

- Each block in the encoder is: Convolution -> >PReLU
- Each block in the decoder is: Transposed convolution -> Dropout (applied to the first 3 blocks) -> ReLU
- There are skip connections between the encoder and decoder (as in the U-Net).

Define the downsampler (encoder):

In [ ]:
OUTPUT_CHANNELS = compNUM

In [ ]:
def downsample(filters, size, apply_batchnorm=True):
    initializer = tf.random_normal_initializer(0., 0.02)

    result = tf.keras.Sequential()
    result.add(
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))
    result.add(tf.keras.layers.PReLU())

    return result

Define the upsampler (decoder):

In [ ]:
def upsample(filters, size, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)

    result = tf.keras.Sequential()
    result.add(
        tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                    padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False))

    if apply_dropout:
        result.add(tf.keras.layers.Dropout(0.5))

    result.add(tf.keras.layers.PReLU())

    return result

Define the generator with the downsampler and the upsampler:

In [ ]:
def Generator():
    inputs = tf.keras.layers.Input(shape=[128, 128, compNUM])

    down_stack = [
        downsample(64, 5, apply_batchnorm=False),  # (batch_size, 64, 64, 64)
        downsample(128, 5),  # (batch_size, 32, 32, 128)
        downsample(256, 5),  # (batch_size, 16, 16, 256)
        downsample(512, 5),  # (batch_size, 8, 8, 512)
        downsample(512, 5),  # (batch_size, 4, 4, 512)
        downsample(512, 5),  # (batch_size, 2, 2, 512)
        downsample(512, 5),  # (batch_size, 1, 1, 512)
    ]

    up_stack = [
        upsample(512, 5, apply_dropout=True),  # (batch_size, 2, 2, 1024)
        upsample(512, 5, apply_dropout=True),  # (batch_size, 4, 4, 1024)
        upsample(512, 5, apply_dropout=True),  # (batch_size, 8, 8, 1024)
        upsample(512, 5),  # (batch_size, 16, 16, 768)
        upsample(256, 5),  # (batch_size, 32, 32, 384)
        upsample(128, 5),  # (batch_size, 64, 64, 192)
    ]
    
    initializer = tf.random_normal_initializer(0., 0.02)
    last = tf.keras.layers.Conv2DTranspose(OUTPUT_CHANNELS, 5,
                                           strides=2,
                                           padding='same',
                                           kernel_initializer=initializer,
                                           activation='tanh')  # (batch_size, 128, 128, 3)

    x = inputs

    # Downsampling through the model
    skips = []
    for down in down_stack:
        x = down(x)
        skips.append(x)

    skips = reversed(skips[:-1])

    # Upsampling and establishing the skip connections
    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = tf.keras.layers.Concatenate()([x, skip])

    x = last(x)

    return tf.keras.Model(inputs=inputs, outputs=x)

Visualize the generator model architecture:

In [ ]:
generator = Generator()
tf.keras.utils.plot_model(generator, show_shapes=True, dpi=64)
# generator.summary()

### Define the generator loss

GANs learn a loss that adapts to the data, while cGANs learn a structured loss that penalizes a possible structure that differs from the network output and the target image, as described in the [pix2pix paper](https://arxiv.org/abs/1611.07004).

- The generator loss is a sigmoid cross-entropy loss of the generated images and an **array of ones**.
- The pix2pix paper also mentions the L1 loss, which is a MAE (mean absolute error) between the generated image and the target image.
- This allows the generated image to become structurally similar to the target image.
- The formula to calculate the total generator loss is `gan_loss + LAMBDA * l1_loss`, where `LAMBDA = 1000`. Original value of 100.

In [ ]:
LAMBDA = 1000

In [ ]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def generator_loss(disc_generated_output, gen_output, target):
    gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)

    # Mean absolute error
    l1_loss = tf.reduce_mean(tf.abs(target - gen_output))

    total_gen_loss = gan_loss + (LAMBDA * l1_loss)

    return total_gen_loss, gan_loss, l1_loss

## Build the discriminator

The discriminator in the pix2pix cGAN is a convolutional PatchGAN classifier—it tries to classify if each image _patch_ is real or not real, as described in the [pix2pix paper](https://arxiv.org/abs/1611.07004).

- Each block in the discriminator is: Convolution -> PReLU.
- The shape of the output after the last layer is `(batch_size, 7, 7, 1)`.
- The discriminator receives 2 inputs: 
    - The input image and the target image, which it should classify as real.
    - The input image and the generated image (the output of the generator), which it should classify as fake.
    - Use `tf.concat([inp, tar], axis=-1)` to concatenate these 2 inputs together.

Let's define the discriminator:

In [ ]:
def Discriminator():
    initializer = tf.random_normal_initializer(0., 0.02)

    inp = tf.keras.layers.Input(shape=[128, 128, OUTPUT_CHANNELS], name='input_image')
    tar = tf.keras.layers.Input(shape=[128, 128, OUTPUT_CHANNELS], name='target_image')

    x = tf.keras.layers.concatenate([inp, tar])  # (batch_size, 128, 128, channels*2)

    down1 = downsample(64, 7, False)(x)  # (batch_size, 64, 64, 64)
    down2 = downsample(128, 7)(down1)  # (batch_size, 32, 32, 128)
    down3 = downsample(256, 7)(down2)  # (batch_size, 16, 16, 256)

    zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3)  # (batch_size, 18, 18, 256)
    conv = tf.keras.layers.Conv2D(512, 4, strides=1,
                                kernel_initializer=initializer,
                                use_bias=False)(zero_pad1)  # (batch_size, 15, 15, 512)

    batchnorm1 = tf.keras.layers.BatchNormalization()(conv)

    leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)

    zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu)  # (batch_size, 17, 17, 512)

    last = tf.keras.layers.Conv2D(1, 4, strides=2,
                                kernel_initializer=initializer)(zero_pad2)  # (batch_size, 7, 7, 1)

    return tf.keras.Model(inputs=[inp, tar], outputs=last)

Visualize the discriminator model architecture:

In [ ]:
discriminator = Discriminator()
tf.keras.utils.plot_model(discriminator, show_shapes=True, dpi=64)
# discriminator.summary()

### Define the discriminator loss

- The `discriminator_loss` function takes 2 inputs: **real images** and **generated images**.
- `real_loss` is a sigmoid cross-entropy loss of the **real images** and an **array of ones(since these are the real images)**.
- `generated_loss` is a sigmoid cross-entropy loss of the **generated images** and an **array of zeros (since these are the fake images)**.
- The `total_loss` is the sum of `real_loss` and `generated_loss`.

In [ ]:
def discriminator_loss(disc_real_output, disc_generated_output):
    real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)

    generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

    total_disc_loss = real_loss + generated_loss

    return total_disc_loss

## Define the optimizers and a checkpoint-saver


In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [ ]:
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

## Generate images

Write a function to plot some images during training.

- Pass images from the test set to the generator.
- The generator will then translate the input image into the output.
- The last step is to plot the predictions and _voila_!

Note: The `training=True` is intentional here since you want the batch statistics, while running the model on the test dataset. If you use `training=False`, you get the accumulated statistics learned from the training dataset (which you don't want).

In [ ]:
def generate_images(model, test_input, tar, plot=True):
    prediction = model(test_input, training=True)
    
    if plot:
        plt.figure(figsize=(15, 15))

        display_list = [test_input[0], tar[0], prediction[0]]
        title = ['Input Image', 'Ground Truth', 'Predicted Image']

        for i in range(3):
            plt.subplot(1, 3, i+1)
            plt.title(title[i])

            # Getting the pixel values in the [0, 1] range to plot
            # and setting the third channel to a constant 0.5
            plt.imshow(tf.concat((display_list[i], 
                                  tf.zeros_like(display_list[i][...,:1])), axis=-1) * 0.5 + 0.5)
            plt.axis('off')
        plt.show()
    return prediction  

## Training

- For each example input generates an output.
- The discriminator receives the `input_image` and the generated image as the first input. The second input is the `input_image` and the `target_image`.
- Next, calculate the generator and the discriminator loss.
- Then, calculate the gradients of loss with respect to both the generator and the discriminator variables(inputs) and apply those to the optimizer.
- Finally, log the losses to TensorBoard.

### Logging 

In [ ]:

# current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
current_time = f'lambda_{LAMBDA}'

summary_writer_train = tf.summary.create_file_writer(
  log_dir + current_time + '/train')
summary_writer_val = tf.summary.create_file_writer(
  log_dir + current_time + '/validation')

### `train_step`, `test_step` and `fit` functions

In [ ]:
@tf.function
def train_step(input_image, target, step):
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        gen_output = generator(input_image, training=True)

        disc_real_output = discriminator([input_image, target], training=True)
        disc_generated_output = discriminator([input_image, gen_output], training=True)

        gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output, gen_output, target)
        disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

    # Gradients
    generator_gradients = gen_tape.gradient(gen_total_loss,
                                            generator.trainable_variables)
    discriminator_gradients = disc_tape.gradient(disc_loss,
                                                 discriminator.trainable_variables)
    # Apply gradients
    generator_optimizer.apply_gradients(zip(generator_gradients,
                                            generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                                discriminator.trainable_variables))
    
    return gen_total_loss, gen_gan_loss, gen_l1_loss, disc_loss, gen_total_loss

In [ ]:
@tf.function
def test_step(input_image, target, step):
    
    gen_output = generator(input_image, training=True)

    disc_real_output = discriminator([input_image, target], training=True)
    disc_generated_output = discriminator([input_image, gen_output], training=True)

    gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output, gen_output, target)
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

    return gen_total_loss, gen_gan_loss, gen_l1_loss, disc_loss, gen_total_loss

In [ ]:
def fit(train_ds, test_ds, epochs):
    tloss_list = []
    dloss_list = []
    gloss_list = []
    
    tlossOLD = 1e10
    for example_input, example_target in test_dataset.take(1):
        pass
    
    start = time.time()
    display.clear_output(wait=True)
    for epoch in tqdm(range(epochs), desc="epoch", position=0):
        lr = generator_optimizer.lr.numpy()
        
        # Traingin for each batch
        for step, (input_image, target) in tqdm(train_ds.enumerate(), 
                                                desc='batch', position=1, leave=False):
            gen_total_loss, gen_gan_loss, gen_l1_loss, disc_loss, gen_total_loss = train_step(input_image, target, step)
        
        with summary_writer_train.as_default():
            tf.summary.scalar('gen_total_loss', gen_total_loss, step=epoch)
            tf.summary.scalar('gen_gan_loss', gen_gan_loss, step=epoch)
            tf.summary.scalar('gen_l1_loss', gen_l1_loss, step=epoch)
            tf.summary.scalar('disc_loss', disc_loss, step=epoch)
            tf.summary.scalar('TOTAL_loss', gen_total_loss + disc_loss, step=epoch)
            tf.summary.scalar('lr', lr, step=epoch)
            
        tloss_list.append(gen_total_loss + disc_loss)
        dloss_list.append(disc_loss)
        gloss_list.append(gen_gan_loss)
        
        # Test loop
        for step, (input_image, target) in test_ds.enumerate():
            gen_total_loss, gen_gan_loss, gen_l1_loss, disc_loss, gen_total_loss = test_step(input_image, target, step)

        with summary_writer_val.as_default():
            tf.summary.scalar('gen_total_loss', gen_total_loss, step=epoch)
            tf.summary.scalar('gen_gan_loss', gen_gan_loss, step=epoch)
            tf.summary.scalar('gen_l1_loss', gen_l1_loss, step=epoch)
            tf.summary.scalar('disc_loss', disc_loss, step=epoch)
            tf.summary.scalar('TOTAL_loss', gen_total_loss + disc_loss, step=epoch)
            
        tloss_test = gen_total_loss + disc_loss

        if tloss_test<tlossOLD:
            old_counter = checkpoint.save_counter.numpy()
            checkpoint.save_counter.assign(0)
            checkpoint.save(file_prefix=checkpoint_prefix)
            tlossOLD = tloss_test
            checkpoint.save_counter.assign(old_counter)
        
        # Check if the Generator-Discriminator losses have stabilized
        if epoch>2:        
            fit_curve = tf.concat((tf.concat(tloss_list, axis=0)[...,tf.newaxis], 
                          tf.concat(gloss_list, axis=0)[...,tf.newaxis], 
                          tf.concat(dloss_list, axis=0)[...,tf.newaxis]), axis=1)
            
            df = pd.DataFrame(fit_curve, columns=['Tot', 'G', 'D'])
            rolling = df.rolling(300).std()
            
            if ((rolling.G.iloc[-1]<rolling.G.max()*0.1) |
                (rolling.D.iloc[-1]<rolling.D.max()*0.1)):
                
                print('Early stopping... saving model.')
                checkpoint.save(file_prefix=checkpoint_prefix)
                break
                        
    return fit_curve

Finally, run the training loop:

In [ ]:
%%time

print('Training...')
fit_curve = fit(train_dataset, val_dataset, epochs=1500)

Interpreting the logs is more subtle when training a GAN (or a cGAN like pix2pix) compared to a simple classification or regression model. Things to look for:

- Check that neither the generator nor the discriminator model has "won". If either the `gen_gan_loss` or the `disc_loss` gets very low, it's an indicator that this model is dominating the other, and you are not successfully training the combined model.
- The value `log(2) = 0.69` is a good reference point for these losses, as it indicates a perplexity of 2 - the discriminator is, on average, equally uncertain about the two options.
- For the `disc_loss`, a value below `0.69` means the discriminator is doing better than random on the combined set of real and generated images.
- For the `gen_gan_loss`, a value below `0.69` means the generator is doing better than random at fooling the discriminator.
- As training progresses, the `gen_l1_loss` should go down.

## Restore the latest checkpoint and test the network

In [ ]:
!ls {checkpoint_dir}

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
# Restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

## Generate some images using the `test set`

In [ ]:
%%time
# Run the trained model on a few examples from the test set
norm = colors.Normalize(vmin=-60, vmax=80)

pred_list = []

for inp, tar in test_dataset:
    pred = generate_images(generator, inp, tar, plot=False)
    pred01 = (pred.numpy()*0.5+0.5)
    tar01 = (tar.numpy()*0.5+0.5)
    
    try:
        pred_list = np.concatenate((pred_list, norm.inverse(pred01)))
    except ValueError:
        pred_list = norm.inverse(pred01)

## Generate some images using the `train set`

In [ ]:
%%time
# Run the trained model on a few examples from the train set
norm = colors.Normalize(vmin=-60, vmax=80)

train_list = []

for inp, tar in train_dataset:
    pred = generate_images(generator, inp, tar, plot=False)
    pred01 = (pred.numpy()*0.5+0.5)
    tar01 = (tar.numpy()*0.5+0.5)
    
    try:
        train_list = np.concatenate((train_list, norm.inverse(pred01)))
    except ValueError:
        train_list = norm.inverse(pred01)

### Save data

In [ ]:
np.savez_compressed(res_dir+f'lambda{LAMBDA}', 
                    train=train_list[...,:OUTPUT_CHANNELS], 
                    test=pred_list[...,:OUTPUT_CHANNELS], 
                    loss=np.array([np.nan]), #legacy for analysis script, not relevant
                    val_loss=np.array([np.nan]), #legacy for analysis script, not relevant
                    hist=None)